In [1]:
import pandas as pd
import requests
from config import nps_key,weather_key
# from control import nps_key,weather_key
pd.options.mode.chained_assignment = None

# Extract

In [3]:
# National Parks API Documentation : https://www.nps.gov/subjects/developer/api-documentation.htm

url = "https://developer.nps.gov/api/v1/parks?"
park = []
params = {'api_key':nps_key,
# 'stateCode':stateCode,
'q':"hiking",
'limit':300}
response = requests.get(url,params=params).json()
park=[]
for x in range(len(response['data'])):

            park_name = response['data'][x]['fullName']
            parkCode = response['data'][x]['parkCode']
            
            street_add = response['data'][x]['addresses'][0]['line1']
            city  = response['data'][x]['addresses'][0]['city']
            state = response['data'][x]['addresses'][0]['stateCode']
            zip_code = response['data'][x]['addresses'][0]['postalCode']
            email = response['data'][x]['contacts']['emailAddresses'][0]['emailAddress']
            lat = response['data'][x]['latitude']
            lng = response['data'][x]['longitude']
            weather_info = response['data'][x]['weatherInfo']
            try:
                fee_car = response['data'][x]['entranceFees'][0]['cost']
                fee_person = response['data'][x]['entranceFees'][2]['cost']
                phone = response['data'][x]['contacts']['phoneNumbers'][0]['phoneNumber']
            except:
                fee_car = 0
                fee_person = 0
                phone = "N/A"
            data_dict = {'park_code':parkCode,
                'park_name': park_name,
            'fee_per_car':fee_car,
            'fee_per_person':fee_person,
            'street_address':street_add,
            'city':city,
            'state':state,
            'zip_code':zip_code,
            'phone_number':phone,
            'email_address':email,
            'lat':lat,
            'lng':lng,
            'Weather Summary':weather_info}
            park.append(data_dict)
park_df=pd.DataFrame(park)
park_df

,park_code,park_name,fee_per_car,fee_per_person,street_address,city,state,zip_code,phone_number,email_address,lat,lng,Weather Summary
0,acad,Acadia National Park,30.00,15.00,25 Visitor Center Road,Bar Harbor,ME,04609,2072883338,acadia_information@nps.gov,44.409286,-68.247501,"Located on Mount Desert Island in Maine, Acadi..."
1,agfo,Agate Fossil Beds National Monument,0,0,301 River Road,Harrison,NE,69346,N/A,agfo_ranger_activities@nps.gov,42.42170419,-103.753886,"Summers can be very warm, high 90's to 100 wit..."
2,alka,Ala Kahakai National Historic Trail,0,0,Superintendent,Kailua-Kona,HI,96740,N/A,aric_arakaki@nps.gov,19.144668109,-155.890734294,Hawaiʻiʻs coast is generally sunny and hot. Be...
3,alfl,Alibates Flint Quarries National Monument,0,0,37084 Alibates Rd.,Potter County,TX,79036,N/A,lamr_interpretation@nps.gov,35.5819662,-101.6717008,The Texas Panhandle experiences a wide variety...
4,alpo,Allegheny Portage Railroad National Historic Site,0,0,110 Federal Park Road,Gallitzin,PA,16641,N/A,alpo_interpretation@nps.gov,40.42977467,-78.57431622,"The park is located in Cambria County, PA, and..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
270,yell,Yellowstone National Park,35.00,20.00,2 Officers Row,Yellowstone National Park,WY,82190,307-344-7381,YELL_Information@nps.gov,44.59824417,-110.5471695,"Yellowstone's weather can vary quite a bit, ev..."
271,york,Yorktown Battlefield Part of Colonial National...,0,0,Yorktown Battlefield,Yorktown,VA,23690,N/A,colo_info@nps.gov,37.2195,-76.4983,"Yorktown, VA climate is warm during summer whe..."
272,yose,Yosemite National Park,35.00,20.00,9039 Village Drive,Yosemite,CA,95389,209/372-0200,yose_web_manager@nps.gov,37.84883288,-119.5571873,"Yosemite National Park covers nearly 1,200 squ..."
273,yuch,Yukon - Charley Rivers National Preserve,0,0,101 Dunkel St,Fairbanks,AK,99701,N/A,fair_information@nps.gov,65.0935608,-142.7960021,The area that Yukon-Charley Rivers encompasses...


In [58]:
weather_df = park_df[['park_code','park_name','city','state','lat','lng']]
weather_df["min_temp"]=""
weather_df["max_temp"]=""
weather_df["humidity"]=""
weather_df["cloudiness"]=""
weather_df["wind_speed"]=""
weather_df["date"]=""


units = 'imperial'
weather_key=weather_key
openweather =  "https://api.openweathermap.org/data/2.5/weather?"


for index,row in weather_df.iterrows():
    park = row["park_name"]
    lat = row['lat']
    lng = row['lng']
    params = {'appid':weather_key,
             'lat':lat,
             'lon':lng,
             'units':units}
    response = requests.get(openweather,params=params).json()
    try:
        min_temp = response["main"]["temp_min"]
        max_temp = response["main"]["temp_max"]
        humidity = response["main"]["humidity"]
        wind = response["wind"]["speed"]
        date = response["dt"]
        clouds = response['clouds']['all']
        weather_df.loc[index,"humidity"]=humidity
        weather_df.loc[index,"min_temp"]=min_temp
        weather_df.loc[index,"max_temp"]=max_temp
        weather_df.loc[index,"humidity"]=humidity
        weather_df.loc[index,"cloudiness"]=clouds
        weather_df.loc[index,"wind_speed"]=wind
        weather_df.loc[index,"date"]=date
    except:
        pass
weather_df

,park_code,park_name,city,state,lat,lng,min_temp,max_temp,humidity,cloudiness,wind_speed,date
0,acad,Acadia National Park,Bar Harbor,ME,44.409286,-68.247501,59.04,65.41,44,0,17,1663264403
1,agfo,Agate Fossil Beds National Monument,Harrison,NE,42.42170419,-103.753886,65.82,66.65,44,70,5.35,1663264403
2,alka,Ala Kahakai National Historic Trail,Kailua-Kona,HI,19.144668109,-155.890734294,77.65,77.65,76,13,4.32,1663264403
3,alfl,Alibates Flint Quarries National Monument,Potter County,TX,35.5819662,-101.6717008,80.37,87.3,52,19,18.86,1663264404
4,alpo,Allegheny Portage Railroad National Historic Site,Gallitzin,PA,40.42977467,-78.57431622,64,71.73,54,13,7.4,1663264404
...,...,...,...,...,...,...,...,...,...,...,...,...
270,yell,Yellowstone National Park,Yellowstone National Park,WY,44.59824417,-110.5471695,47.44,50.38,93,57,3.44,1663264485
271,york,Yorktown Battlefield Part of Colonial National...,Yorktown,VA,37.2195,-76.4983,76.84,81.86,61,40,10.36,1663264486
272,yose,Yosemite National Park,Yosemite,CA,37.84883288,-119.5571873,47.08,52.75,25,49,8.05,1663264486
273,yuch,Yukon - Charley Rivers National Preserve,Fairbanks,AK,65.0935608,-142.7960021,40.71,40.71,91,97,3.27,1663264486


In [87]:
weather_df.to_csv('cleaned/weather_data.csv',index=False)

# Transform

In [4]:
# Replacing national park names with extra characters
park_df = park_df.replace({'Hawaiʻi Volcanoes National Park':'Hawaii Volcanoes National Park','Haleakalā National Park':'Haleakala National Park','Tumacácori National Historical Park':'Tumacacori National Historical Park'})

In [5]:
# All Trails pulled from 
# https://www.kaggle.com/code/jessicaqjiang/all-trails-in-np-analysis/data
all_trails = pd.read_csv('Working/Resources/AllTrails data - nationalpark.csv')
all_trails=all_trails.reset_index(drop=True)
all_trails

,trail_id,name,area_name,city_name,state_name,country_name,_geoloc,popularity,length,elevation_gain,difficulty_rating,route_type,visitor_usage,avg_rating,num_reviews,features,activities,units
0,10020048,Harding Ice Field Trail,Kenai Fjords National Park,Seward,Alaska,United States,"{'lat': 60.18852, 'lng': -149.63156}",24.8931,15610.598,1161.8976,5,out and back,3.0,5.0,423,"['dogs-no', 'forest', 'river', 'views', 'water...","['birding', 'camping', 'hiking', 'nature-trips...",i
1,10236086,Mount Healy Overlook Trail,Denali National Park,Denali National Park,Alaska,United States,"{'lat': 63.73049, 'lng': -148.91968}",18.0311,6920.162,507.7968,3,out and back,1.0,4.5,260,"['dogs-no', 'forest', 'views', 'wild-flowers',...","['birding', 'camping', 'hiking', 'nature-trips...",i
2,10267857,Exit Glacier Trail,Kenai Fjords National Park,Seward,Alaska,United States,"{'lat': 60.18879, 'lng': -149.631}",17.7821,2896.812,81.9912,1,out and back,3.0,4.5,224,"['dogs-no', 'partially-paved', 'views', 'wildl...","['hiking', 'walking']",i
3,10236076,Horseshoe Lake Trail,Denali National Park,Denali National Park,Alaska,United States,"{'lat': 63.73661, 'lng': -148.915}",16.2674,3379.614,119.7864,1,loop,2.0,4.5,237,"['dogs-no', 'forest', 'lake', 'kids', 'views',...","['birding', 'hiking', 'nature-trips', 'trail-r...",i
4,10236082,Triple Lakes Trail,Denali National Park,Denali National Park,Alaska,United States,"{'lat': 63.73319, 'lng': -148.89682}",12.5935,29772.790,1124.7120,5,out and back,1.0,4.5,110,"['dogs-no', 'lake', 'views', 'wild-flowers', '...","['birding', 'fishing', 'hiking', 'nature-trips...",i
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3308,10008302,Silversword Loop Via Halemau'u Trail,Haleakala National Park,Kula,Maui,Hawaii,"{'lat': 20.75275, 'lng': -156.22884}",9.3861,20116.750,1105.8144,5,loop,2.0,4.5,43,"['dogs-no', 'views', 'wild-flowers']","['birding', 'hiking', 'nature-trips']",m
3309,10236001,Keonehe'ehe'e Trail,Haleakala National Park,Kula,Maui,Hawaii,"{'lat': 20.714480000000002, 'lng': -156.25072}",9.1555,28324.384,1171.9560,5,out and back,2.0,5.0,22,"['dogs-no', 'views', 'wildlife']","['backpacking', 'camping', 'hiking']",m
3310,10258707,Red Hill Overlook Summit Trail,Haleakala National Park,Kula,Maui,Hawaii,"{'lat': 20.71007, 'lng': -156.25357}",8.5066,321.868,3.9624,1,out and back,NaN,4.5,31,"['dogs-no', 'kids', 'views']","['hiking', 'walking']",m
3311,10014989,Kaupo Trail,Haleakala National Park,Kula,Maui,Hawaii,"{'lat': 20.64981, 'lng': -156.137}",8.3240,19312.080,1670.9136,5,out and back,1.0,4.0,8,"['dogs-no', 'views', 'wildlife']",['hiking'],m


In [6]:
# Some National Parks are listed differently in the All Trails Dataset; need to rename
all_trails = all_trails.replace({'Black Canyon of the Gunnison National Park':'Black Canyon Of The Gunnison National Park',
                          'Congaree National Park Wilderness':'Congaree National Park,','Denali National Park':'Denali National Park & Preserve',
                          'Glacier Bay National Park':'Glacier Bay National Park & Preserve','Great Sand Dunes National Park and Preserve':'Great Sand Dunes National Park & Preserve',
                          'Katmai National Park':'Katmai National Park & Preserve','Kings Canyon National Park':'Sequoia & Kings Canyon National Parks',
                          'Sequoia National Park':'Sequoia & Kings Canyon National Parks','Redwood National Park':'Redwood National and State Parks'})

In [7]:
# Inserting Park Code as a sort of Foreign Key
park_code = park_df[["park_name","park_code"]]
all_trails['park_code']=''

for index,row in all_trails.iterrows():
    park = row['area_name']
    try:
        code=park_code.loc[(park_code['park_name']==park),['park_code']].values[0][0]
        all_trails.iloc[index,18]=code
    except:
        all_trails.iloc[index,18]='look'

In [8]:
# Some national parks in All Trails do not appear in the National Parks API; filter out and save off
look_df=all_trails.loc[all_trails['park_code']=='look']
look_df.to_csv('Working/Resources/cleaned/trails_without_code.csv')
look_df

,trail_id,name,area_name,city_name,state_name,country_name,_geoloc,popularity,length,elevation_gain,difficulty_rating,route_type,visitor_usage,avg_rating,num_reviews,features,activities,units,park_code
1190,10035361,Fort Pickens Trail,Fort Pickens National Park,Gulf Breeze,Florida,United States,"{'lat': 30.31858, 'lng': -87.24114}",7.8388,13196.588,26.8224,1,out and back,NaN,4.5,35,"['beach', 'dogs-no']","['hiking', 'mountain-biking', 'trail-running',...",i,look
1194,10034730,Biscayne National Park Walk,Biscayne National Park,Miami,Florida,United States,"{'lat': 25.53849, 'lng': -80.33091}",5.8949,15932.466,0.9144,1,out and back,1.0,4.0,16,"['beach', 'dogs-no', 'kids', 'paved', 'views',...","['birding', 'fishing', 'nature-trips', 'paddle...",i,look
1197,10025862,Fort Jefferson Loop,Dry Tortugas National Park,Key West,Florida,United States,"{'lat': 24.62723, 'lng': -82.87242}",5.1776,804.670,0.9144,1,loop,3.0,4.5,10,"['beach', 'kids', 'views', 'wildlife']","['birding', 'camping', 'hiking', 'nature-trips...",i,look
1200,10111232,Blackbird Marsh Trail,Fort Pickens National Park,Gulf Breeze,Florida,United States,"{'lat': 30.31943, 'lng': -87.26118}",4.9089,2574.944,2.7432,1,loop,NaN,4.5,14,"['dogs-leash', 'kids', 'views', 'wildlife']","['birding', 'hiking', 'walking']",i,look
1202,10294502,Black Creek Canal Trail,Biscayne National Park,Homestead,Florida,United States,"{'lat': 25.54017, 'lng': -80.32772}",4.3565,2735.878,0.9144,1,out and back,2.0,4.0,16,"['beach', 'dogs-no', 'forest', 'kids', 'views'...","['hiking', 'nature-trips', 'walking']",i,look
1204,10347885,Jetty Trail,Biscayne National Park,Homestead,Florida,United States,"{'lat': 25.46415, 'lng': -80.3356}",3.9733,1448.406,0.0000,1,out and back,2.0,4.0,14,"['kids', 'views', 'wild-flowers', 'wildlife']","['birding', 'hiking', 'nature-trips', 'walking']",i,look
1216,10294501,Boca Chita Trail,Biscayne National Park,Homestead,Florida,United States,"{'lat': 25.52443, 'lng': -80.17485}",1.9961,1287.472,0.9144,1,loop,1.0,3.0,3,"['beach', 'dogs-no', 'forest', 'historic-site'...","['hiking', 'walking']",i,look
1219,10259207,Clayton County International Park Trail,"Clayton Co International Park, Jonesboro GA",Jonesboro,Georgia,United States,"{'lat': 33.5312, 'lng': -84.31568}",4.7291,2574.944,36.8808,3,loop,1.0,4.0,26,"['dogs-leash', 'forest', 'kids', 'views', 'wil...","['birding', 'hiking', 'mountain-biking', 'natu...",i,look
1454,10265487,St. Louis Riverfront Trail,Gateway Arch National Park,St. Louis,Missouri,United States,"{'lat': 38.63514, 'lng': -90.18118}",6.1208,34439.876,109.7280,1,loop,2.0,4.0,25,"['ada', 'city-walk', 'dogs-leash', 'paved', 'r...","['mountain-biking', 'road-biking', 'trail-runn...",i,look
1455,10030475,Jefferson National Expansion Memorial,Gateway Arch National Park,St. Louis,Missouri,United States,"{'lat': 38.62327, 'lng': -90.18743}",4.4919,2574.944,20.7264,1,loop,1.0,4.0,17,"['ada', 'city-walk', 'dogs-leash', 'kids', 'ri...",['walking'],i,look


In [9]:
all_trails_cleaned = all_trails.loc[all_trails['park_code']!='look',:]
# Rename the column with trail name
all_trails_cleaned.rename(columns= {"name": "trail"}, inplace = True)

# Drop the unwanted columns
# Upon investigation all units are in meters regardless of the input in the units column
all_trails_cleaned = all_trails_cleaned.drop(['country_name','visitor_usage','units'], axis = 1)

# Make the trail_id the index
all_trails_cleaned=all_trails_cleaned.set_index('trail_id')

In [10]:
alltrails_filtered =all_trails_cleaned[all_trails_cleaned['avg_rating'] >= 2]
alltrails_filtered = all_trails_cleaned[all_trails_cleaned['num_reviews'] >= 10]
alltrails_filtered.sort_values(by='popularity', inplace=True, ascending=False)
alltrails_filtered.to_csv('cleaned/trails.csv')
alltrails_filtered

,trail,area_name,city_name,state_name,_geoloc,popularity,length,elevation_gain,difficulty_rating,route_type,avg_rating,num_reviews,features,activities,park_code
trail_id,,,,,,,,,,,,,,,
10006571,Angels Landing Trail,Zion National Park,Springdale,Utah,"{'lat': 37.25928, 'lng': -112.95166}",84.6229,6598.294,492.8616,5,out and back,5.0,3903,"['dogs-no', 'partially-paved', 'river', 'views...","['hiking', 'nature-trips', 'trail-running']",zion
10026705,Emerald Lake Trail,Rocky Mountain National Park,Estes Park,Colorado,"{'lat': 40.31177, 'lng': -105.64533}",83.0341,4988.954,215.7984,3,out and back,4.5,2991,"['dogs-no', 'forest', 'lake', 'kids', 'partial...","['birding', 'fishing', 'hiking', 'nature-trips...",romo
10026910,Vernal and Nevada Falls via the Mist Trail,Yosemite National Park,Yosemite Valley,California,"{'lat': 37.73495, 'lng': -119.56619}",68.1488,10299.776,667.8168,3,loop,5.0,1869,"['dogs-no', 'forest', 'partially-paved', 'rive...","['birding', 'hiking', 'nature-trips', 'walking']",yose
10235688,Sky Pond via Glacier Gorge Trail,Rocky Mountain National Park,Estes Park,Colorado,"{'lat': 40.31052, 'lng': -105.64032}",65.4770,13035.654,537.9720,5,out and back,5.0,1599,"['dogs-no', 'forest', 'lake', 'river', 'views'...","['birding', 'hiking', 'nature-trips', 'snowsho...",romo
10012464,Delicate Arch Trail,Arches National Park,Moab,Utah,"{'lat': 38.73568, 'lng': -109.5205}",63.6748,4988.954,186.8424,3,out and back,5.0,2332,"['dogs-no', 'kids', 'partially-paved', 'views'...","['birding', 'hiking', 'nature-trips', 'rock-cl...",arch
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10278538,Kayenta Trail to the Emerald Pools and Zion Lo...,Zion National Park,Springdale,Utah,"{'lat': 37.25918, 'lng': -112.95125}",1.6750,6920.162,313.9440,3,out and back,4.0,26,"['dogs-no', 'lake', 'kids', 'river', 'views', ...","['hiking', 'nature-trips', 'walking']",zion
10111637,Careys Castle Trail [CLOSED],Joshua Tree National Park,Mecca,California,"{'lat': 33.69963, 'lng': -115.6785}",1.5663,12070.050,388.9248,3,out and back,3.5,10,"['dogs-no', 'views', 'wild-flowers', 'wildlife']","['birding', 'hiking', 'nature-trips', 'trail-r...",jotr
10235829,Old Carriage Trail [CLOSED],Cuyahoga Valley National Park,Brecksville,Ohio,"{'lat': 41.28957, 'lng': -81.56437}",1.4010,8529.502,105.7656,3,loop,3.5,41,"['dogs-leash', 'forest', 'kids', 'partially-pa...","['hiking', 'nature-trips', 'trail-running', 'w...",cuva


In [95]:
summary_table = pd.DataFrame({'num_of_trails':alltrails_filtered.groupby('park_code').count()['trail'],
                'avg_difficulty':alltrails_filtered.groupby('park_code').mean()['difficulty_rating'],
                'avg_rating':alltrails_filtered.groupby('park_code').mean()['avg_rating']})

summary_table = summary_table.reset_index(drop=False)
summary_table

,park_code,num_of_trails,avg_difficulty,avg_rating
0,acad,133,2.879699,4.473684
1,arch,34,1.941176,4.397059
2,badl,9,2.333333,4.166667
3,bibe,44,2.500000,4.238636
4,blca,20,2.100000,4.350000
5,brca,28,2.500000,4.428571
6,cany,38,2.684211,4.407895
7,care,29,2.655172,4.465517
8,cave,2,1.000000,4.500000
9,chis,11,2.636364,4.454545


In [96]:
aggregate_park = pd.merge(park_df,summary_table,how = 'left', on='park_code')
aggregate_park = aggregate_park.fillna(0)
aggregate_park.to_csv('cleaned/National_Parks.csv',index=False)
aggregate_park

,park_code,park_name,fee_per_car,fee_per_person,street_address,city,state,zip_code,phone_number,email_address,lat,lng,Weather Summary,num_of_trails,avg_difficulty,avg_rating
0,acad,Acadia National Park,30.00,15.00,25 Visitor Center Road,Bar Harbor,ME,04609,2072883338,acadia_information@nps.gov,44.409286,-68.247501,"Located on Mount Desert Island in Maine, Acadi...",133.0,2.879699,4.473684
1,agfo,Agate Fossil Beds National Monument,0,0,301 River Road,Harrison,NE,69346,N/A,agfo_ranger_activities@nps.gov,42.42170419,-103.753886,"Summers can be very warm, high 90's to 100 wit...",0.0,0.000000,0.000000
2,alka,Ala Kahakai National Historic Trail,0,0,Superintendent,Kailua-Kona,HI,96740,N/A,aric_arakaki@nps.gov,19.144668109,-155.890734294,Hawaiʻiʻs coast is generally sunny and hot. Be...,0.0,0.000000,0.000000
3,alfl,Alibates Flint Quarries National Monument,0,0,37084 Alibates Rd.,Potter County,TX,79036,N/A,lamr_interpretation@nps.gov,35.5819662,-101.6717008,The Texas Panhandle experiences a wide variety...,0.0,0.000000,0.000000
4,alpo,Allegheny Portage Railroad National Historic Site,0,0,110 Federal Park Road,Gallitzin,PA,16641,N/A,alpo_interpretation@nps.gov,40.42977467,-78.57431622,"The park is located in Cambria County, PA, and...",0.0,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
270,yell,Yellowstone National Park,35.00,20.00,2 Officers Row,Yellowstone National Park,WY,82190,307-344-7381,YELL_Information@nps.gov,44.59824417,-110.5471695,"Yellowstone's weather can vary quite a bit, ev...",123.0,2.219512,4.341463
271,york,Yorktown Battlefield Part of Colonial National...,0,0,Yorktown Battlefield,Yorktown,VA,23690,N/A,colo_info@nps.gov,37.2195,-76.4983,"Yorktown, VA climate is warm during summer whe...",0.0,0.000000,0.000000
272,yose,Yosemite National Park,35.00,20.00,9039 Village Drive,Yosemite,CA,95389,209/372-0200,yose_web_manager@nps.gov,37.84883288,-119.5571873,"Yosemite National Park covers nearly 1,200 squ...",127.0,3.440945,4.590551
273,yuch,Yukon - Charley Rivers National Preserve,0,0,101 Dunkel St,Fairbanks,AK,99701,N/A,fair_information@nps.gov,65.0935608,-142.7960021,The area that Yukon-Charley Rivers encompasses...,0.0,0.000000,0.000000


# Load

In [2]:
from sqlalchemy import create_engine
from config import pg_key
# from control import nps_key,weather_key

In [3]:
protocol = 'postgresql'
username = 'postgres'
password = pg_key
host = 'localhost'
port = 5432

In [99]:
alltrails_filtered = pd.read_csv('cleaned/trails.csv')
weather_data = pd.read_csv('cleaned/weather_data.csv')
aggregate_park = pd.read_csv('cleaned/National_Parks.csv')

In [4]:
database_name = 'National_Parks_DB'
rds_connection_string = f'{protocol}://{username}:{password}@{host}:{port}/{database_name}'
engine = create_engine(rds_connection_string)

In [101]:
alltrails_filtered.to_sql(name='alltrails', con=engine, if_exists='append', index=False)

83

In [102]:
weather_data.to_sql(name='weather_data', con=engine, if_exists='append', index=False)

275

In [103]:
aggregate_park.to_sql(name='national_parks', con=engine, if_exists='append', index=False)

275

In [105]:
pd.read_sql_query('select * from alltrails', con=engine).head()

,trail_id,trail,area_name,city_name,state_name,_geoloc,popularity,length,elevation_gain,difficulty_rating,route_type,avg_rating,num_reviews,features,activities,park_code
0,10006571,Angels Landing Trail,Zion National Park,Springdale,Utah,"{'lat': 37.25928, 'lng': -112.95166}",84.6229,6598.294,492.8616,5,out and back,5.0,3903,"['dogs-no', 'partially-paved', 'river', 'views...","['hiking', 'nature-trips', 'trail-running']",zion
1,10026705,Emerald Lake Trail,Rocky Mountain National Park,Estes Park,Colorado,"{'lat': 40.31177, 'lng': -105.64533}",83.0341,4988.954,215.7984,3,out and back,4.5,2991,"['dogs-no', 'forest', 'lake', 'kids', 'partial...","['birding', 'fishing', 'hiking', 'nature-trips...",romo
2,10026910,Vernal and Nevada Falls via the Mist Trail,Yosemite National Park,Yosemite Valley,California,"{'lat': 37.73495, 'lng': -119.56619}",68.1488,10299.776,667.8168,3,loop,5.0,1869,"['dogs-no', 'forest', 'partially-paved', 'rive...","['birding', 'hiking', 'nature-trips', 'walking']",yose
3,10235688,Sky Pond via Glacier Gorge Trail,Rocky Mountain National Park,Estes Park,Colorado,"{'lat': 40.31052, 'lng': -105.64032}",65.4770,13035.654,537.9720,5,out and back,5.0,1599,"['dogs-no', 'forest', 'lake', 'river', 'views'...","['birding', 'hiking', 'nature-trips', 'snowsho...",romo
4,10012464,Delicate Arch Trail,Arches National Park,Moab,Utah,"{'lat': 38.73568, 'lng': -109.5205}",63.6748,4988.954,186.8424,3,out and back,5.0,2332,"['dogs-no', 'kids', 'partially-paved', 'views'...","['birding', 'hiking', 'nature-trips', 'rock-cl...",arch


In [106]:
pd.read_sql_query('select * from national_parks', con=engine).head()

,park_code,park_name,fee_per_car,fee_per_person,street_address,city,state,zip_code,phone_number,email_address,lat,lng,Weather Summary,num_of_trails,avg_difficulty,avg_rating
0,acad,Acadia National Park,30.0,15.0,25 Visitor Center Road,Bar Harbor,ME,04609,2072883338,acadia_information@nps.gov,44.409286,-68.247501,"Located on Mount Desert Island in Maine, Acadi...",133.0,2.879699,4.473684
1,agfo,Agate Fossil Beds National Monument,0.0,0.0,301 River Road,Harrison,NE,69346,None,agfo_ranger_activities@nps.gov,42.421704,-103.753886,"Summers can be very warm, high 90's to 100 wit...",0.0,0.000000,0.000000
2,alka,Ala Kahakai National Historic Trail,0.0,0.0,Superintendent,Kailua-Kona,HI,96740,None,aric_arakaki@nps.gov,19.144668,-155.890734,Hawaiʻiʻs coast is generally sunny and hot. Be...,0.0,0.000000,0.000000
3,alfl,Alibates Flint Quarries National Monument,0.0,0.0,37084 Alibates Rd.,Potter County,TX,79036,None,lamr_interpretation@nps.gov,35.581966,-101.671701,The Texas Panhandle experiences a wide variety...,0.0,0.000000,0.000000
4,alpo,Allegheny Portage Railroad National Historic Site,0.0,0.0,110 Federal Park Road,Gallitzin,PA,16641,None,alpo_interpretation@nps.gov,40.429775,-78.574316,"The park is located in Cambria County, PA, and...",0.0,0.000000,0.000000


In [107]:
pd.read_sql_query('select * from weather_data', con=engine).head()

,park_code,park_name,city,state,lat,lng,min_temp,max_temp,humidity,cloudiness,wind_speed,date
0,acad,Acadia National Park,Bar Harbor,ME,44.409286,-68.247501,59.04,65.41,44,0,17.00,1663264403
1,agfo,Agate Fossil Beds National Monument,Harrison,NE,42.421704,-103.753886,65.82,66.65,44,70,5.35,1663264403
2,alka,Ala Kahakai National Historic Trail,Kailua-Kona,HI,19.144668,-155.890734,77.65,77.65,76,13,4.32,1663264403
3,alfl,Alibates Flint Quarries National Monument,Potter County,TX,35.581966,-101.671701,80.37,87.30,52,19,18.86,1663264404
4,alpo,Allegheny Portage Railroad National Historic Site,Gallitzin,PA,40.429775,-78.574316,64.00,71.73,54,13,7.40,1663264404


# Queries

In [5]:
# National Parks to visit based on Ideal Weather Conditions
pd.read_sql_query('select n.park_code,n.park_name,t.trail from weather_data \
                  as w JOIN national_parks as n ON w.park_code = n.park_code \
                  JOIN alltrails as t on n.park_code=t.park_code \
                  WHERE w.Min_Temp>70 and w.Max_Temp<80 and w.Cloudiness=0 and n.num_of_trails>0', con=engine)

,park_code,park_name,trail
0,cany,Canyonlands National Park,Mesa Arch Trail
1,cany,Canyonlands National Park,Grand View Point Trail
2,cany,Canyonlands National Park,Chesler Park Loop Trail
3,cany,Canyonlands National Park,False Kiva Trail
4,cany,Canyonlands National Park,Upheaval Dome via Crater View Trail
5,cany,Canyonlands National Park,Aztec Butte Trail
6,cany,Canyonlands National Park,Druid Arch Trail
7,cany,Canyonlands National Park,Shafer Trail
8,cany,Canyonlands National Park,White Rim Overlook Trail
9,cany,Canyonlands National Park,Syncline Loop


In [116]:
# Display trails from the National Park with the highest avg_difficulty, there is only one trail
pd.read_sql_query('select t.trail,t.park_code,n.park_name,t.length,t.elevation_gain from alltrails \
                  as t JOIN national_parks as n ON t.park_code = n.park_code \
                  WHERE n.avg_difficulty = (SELECT max(avg_difficulty) FROM national_parks)', con=engine)

,trail,park_code,park_name,length,elevation_gain
0,Greenstone Ridge Trail: Windigo to Tobin Harbor,isro,Isle Royale National Park,63568.93,1216.7616


In [110]:
# Display top 10 most popular trails
pd.read_sql_query('select trail,area_name,popularity FROM alltrails', con=engine).head(10)

,trail,area_name,popularity
0,Angels Landing Trail,Zion National Park,84.6229
1,Emerald Lake Trail,Rocky Mountain National Park,83.0341
2,Vernal and Nevada Falls via the Mist Trail,Yosemite National Park,68.1488
3,Sky Pond via Glacier Gorge Trail,Rocky Mountain National Park,65.4770
4,Delicate Arch Trail,Arches National Park,63.6748
5,The Zion Narrows Riverside Walk,Zion National Park,62.5499
6,Alum Cave Trail to Mount LeConte,Great Smoky Mountains National Park,61.3755
7,Old Rag Mountain Loop,Shenandoah National Park,59.6754
8,Gem Lake Trail,Rocky Mountain National Park,54.6336
9,Upper Yosemite Falls Trail,Yosemite National Park,53.6280


In [111]:
#Display top 10 highest rated trails
pd.read_sql_query('select t.trail,t.park_code,n.park_name,t.length,t.elevation_gain,t.avg_rating from alltrails \
                  as t JOIN national_parks as n ON t.park_code = n.park_code \
                  WHERE t.avg_rating >4 ORDER BY t.avg_rating DESC' , con=engine).head(10)

,trail,park_code,park_name,length,elevation_gain,avg_rating
0,Nelson Lake Trail,yose,Yosemite National Park,17863.674,781.8120,5.0
1,Hunter Peak,gumo,Guadalupe Mountains National Park,12552.852,809.8536,5.0
2,Bunsen Peak Loop,yell,Yellowstone National Park,16093.400,774.8016,5.0
3,John Muir Trail: Tuolumne to Reds Meadow,yose,Yosemite National Park,52464.484,1644.7008,5.0
4,Dawson Pass and Pitamakan Pass Loop,glac,Glacier National Park,27680.648,1109.7768,5.0
5,East Mesa to Observation Point via Stave Sprin...,zion,Zion National Park,18024.608,702.8688,5.0
6,Rim-to-Rim: North Kaibab to South Kaibab Trail...,grca,Grand Canyon National Park,31060.262,1666.9512,5.0
7,North Kaibab Trail to Redwall Bridge,grca,Grand Canyon National Park,11104.446,801.9288,5.0
8,Hermit Trail to Dripping Springs,grca,Grand Canyon National Park,8529.502,610.8192,5.0
9,Grand Teton National Park Road Trip,grte,Grand Teton National Park,65661.072,456.8952,5.0


In [12]:
#Ideal less 60 degrees, easy trails
pd.read_sql_query('select t.trail,t.park_code,n.park_name,n.avg_difficulty FROM alltrails \
        as t JOIN national_parks as n ON t.park_code = n.park_code \
        JOIN weather_data as w on n.park_code=w.park_code \
        WHERE n.avg_difficulty < 2.5 AND w.min_temp < 60',con=engine)

,trail,park_code,park_name,avg_difficulty
0,"Mystic Falls, Fairy Creek and Little Firehole ...",yell,Yellowstone National Park,2.219512
1,Garfield Peak Trail,crla,Crater Lake National Park,2.111111
2,Grand Prismatic Hot Spring,yell,Yellowstone National Park,2.219512
3,Upper Geyser Basin and Old Faithful Observatio...,yell,Yellowstone National Park,2.219512
4,Cleetwood Cove Trail,crla,Crater Lake National Park,2.111111
...,...,...,...,...
148,Jonesville Trail,dena,Denali National Park & Preserve,2.454545
149,Gneiss Creek Trail,yell,Yellowstone National Park,2.219512
150,Rescue Creek Trail,yell,Yellowstone National Park,2.219512
151,Roadside Trail,dena,Denali National Park & Preserve,2.454545


In [13]:
# Top 10 Highest Elevation Gain
pd.read_sql_query('select t.trail,t.park_code,n.park_name,t.length,t.elevation_gain,t.avg_rating from alltrails \
                  as t JOIN national_parks as n ON t.park_code = n.park_code \
                  ORDER BY t.elevation_gain DESC LIMIT 10',con=engine)

,trail,park_code,park_name,length,elevation_gain,avg_rating
0,John Muir Trail via Yosemite Valley,yose,Yosemite National Park,326856.954,14029.9440,4.5
1,The Wonderland Trail,mora,Mount Rainier National Park,154818.508,7723.9368,4.5
2,Appalachian Trail: Fontana Dam to Davenport Gap,grsm,Great Smoky Mountains National Park,121666.104,5330.9520,5.0
3,Olympic Peninsula Loop Drive,olym,Olympic National Park,529794.728,5096.8656,4.5
4,High Sierra Trail: Crescent Meadow to Whitney ...,seki,Sequoia & Kings Canyon National Parks,94951.060,4716.7800,5.0
5,Maah Daah Hey Trail,thro,Theodore Roosevelt National Park,229009.082,4614.9768,4.5
6,John Muir Trail in Kings Canyon,seki,Sequoia & Kings Canyon National Parks,134701.758,4503.7248,5.0
7,Skyline Drive: Waynesboro to Front Royal,shen,Shenandoah National Park,168497.898,3669.7920,4.5
8,Skyline Drive: Front Royal to Waynesboro,shen,Shenandoah National Park,168658.832,3607.9176,5.0
9,South Kaibab to North Kaibab to Bright Angel T...,grca,Grand Canyon National Park,65822.006,3578.9616,5.0


In [16]:
# Average Elevation Gain by State
pd.read_sql_query('select n.state,avg(t.length) "Average Length",avg(t.elevation_gain) "Average Elevation Gain" from alltrails \
                  as t JOIN national_parks as n ON t.park_code = n.park_code \
                  GROUP BY n.state\
                  ORDER BY avg(t.elevation_gain) DESC',con=engine)

,state,Average Length,Average Elevation Gain
0,MI,63568.930000,1216.761600
1,WA,15902.059115,680.695653
2,TN,15210.485845,651.431695
3,AZ,14173.102780,577.185295
4,MT,16417.254840,573.024000
5,CA,14914.679041,571.564780
6,NV,11288.370571,550.403486
7,VA,12854.933033,496.776531
8,ND,22541.488933,442.894720
9,TX,15490.579424,410.550102


In [58]:
# Average Length and Average Elevation across All Trails
pd.read_sql_query('select '"'Average Across All Trails'"' as "Description", \
avg(t.length) as "Average Length",\
avg(t.elevation_gain) as "Average Elevation Gain" \
from alltrails as t JOIN national_parks as n ON t.park_code = n.park_code',con=engine)

,Description,Average Length,Average Elevation Gain
0,Average Across All Trails,12923.162447,468.370049
